In [1]:
import time
from termcolor import colored
import torch
import torch.autograd.profiler as profiler

from modules.Dcls2d import Dcls2d
from modules.Dcls2dFull import Dcls2dFull


assert torch.cuda.is_available()
cuda_device = torch.device("cuda")  # device object representing GPU

batch=1
img_size = (40,40)
in_channels = 1
out_channels = 1
kernel_size = (7,9)
dilation = (3,5)
stride = (1,1)
padding = (1,1)
groups = 1
bias = False

m = torch.nn.Conv2d(in_channels=in_channels,
              out_channels=out_channels,
              kernel_size=kernel_size,
              dilation=dilation,
              stride=stride,
              padding=padding,
              groups=groups,
              bias=bias).to(cuda_device)



o = Dcls2dFull(in_channels=in_channels,
              out_channels=out_channels,
              kernel_size=kernel_size,
              dilation=dilation,
              stride=stride,
              padding=padding,
              groups=groups,
              bias=bias).to(cuda_device)



height_out = (img_size[0] + 2 * padding[0] - dilation[0] * (kernel_size[0] - 1) - 1) / stride[0] + 1;
width_out = (img_size[1]  + 2 * padding[1] - dilation[1] * (kernel_size[1] - 1) - 1) / stride[1] + 1;
back_truth = torch.nn.Parameter(
                      torch.rand((batch, out_channels, int(height_out),int(width_out)),device=cuda_device),
                      requires_grad = True)

X1 = torch.nn.Parameter(
                      torch.rand((batch, in_channels, img_size[0], img_size[1]),device=cuda_device),
                      requires_grad = True)
m.weight = torch.nn.Parameter(
                      torch.rand((out_channels, in_channels//groups, *kernel_size),device=cuda_device),
                      requires_grad = True)
X3 = torch.nn.Parameter(
                      torch.rand((batch, in_channels, img_size[0], img_size[1]),device=cuda_device),
                      requires_grad = True)
o.weight = torch.nn.Parameter(
                      torch.rand((out_channels, in_channels//groups, *kernel_size),device=cuda_device),
                      requires_grad = True)

o.weight.data = m.weight.data
X3.data = X1.data

var1 = (m(X1) - back_truth).norm()
var3 = (o(X3) - back_truth).norm()


var1.backward();
var3.backward();

SyntaxError: invalid syntax (Dcls2dFull.py, line 25)

In [ ]:
print(X1.size())
print(m.weight.size())
print(o.weight.size())
print(m(X1).size())
print(o(X3).size())

print((torch.abs(m(X1) - o(X3))/torch.abs(m(X1))).max())


print((torch.abs(m.weight.grad - o.weight.grad)/torch.abs(m.weight.grad)).max())



In [ ]:
print(o.P1*dilation[0] * kernel_size[0]*dilation[0] * kernel_size[0])
print(o.P1)
print(o.P1.grad)

In [ ]:
X1 = torch.nn.Parameter(
                      torch.tensor([[[[1., 2., 3., 4.],
                                    [5., 6., 7., 8.], 
                                    [9., 10., 11., 12.],
                                    [13., 14., 15., 16.]]]],device=cuda_device),
                      requires_grad = True) 

X3 = torch.nn.Parameter(
                      torch.tensor([[[[1., 2., 3., 4.],
                                    [5., 6., 7., 8.], 
                                    [9., 10., 11., 12.],
                                    [13., 14., 15., 16.]]]],device=cuda_device),
                      requires_grad = True) 
m.weight = torch.nn.Parameter(
                      torch.tensor([[[[20., 40., 60],
                                      [70., 80., 90],
                                      [100., 120., 140]]]],device=cuda_device),
                      requires_grad = True)

o.weight = torch.nn.Parameter(
                      torch.tensor([[[[20., 40., 60],
                                      [70., 80., 90],
                                      [100., 120., 140]]]],device=cuda_device),
                      requires_grad = True)